# spiral_builder

In [ ]:
open file_system_operators
open rust_operators
open sm'_operators

()



In [ ]:
//// test

open testing

()



## types

In [ ]:
inl types () =
    env.types ()
    file_system.types ()
    runtime.types ()
    rust.types ()
    sm'.types ()

()



## get_command_args

In [ ]:
inl get_command_args () =
    {
        file = "file"
        package_dir = "package-dir"
        args = "args"
        trace_level = "trace-level"
    }

()



## get_command

In [ ]:
let get_command () =
    ##"command"
    |> runtime.new_command
    |> runtime.command_arg (
        runtime.new_arg ##(get_command_args () .file)
        |> runtime.arg_short 'f'
        |> runtime.arg_long ##(get_command_args () .file)
        |> runtime.arg_required true
    )
    |> runtime.command_arg (
        runtime.new_arg ##(get_command_args () .package_dir)
        |> runtime.arg_short 'p'
        |> runtime.arg_long ##(get_command_args () .package_dir)
        |> runtime.arg_required true
    )
    |> runtime.command_arg (
        runtime.new_arg ##(get_command_args () .args)
        |> runtime.arg_short 'a'
        |> runtime.arg_long ##(get_command_args () .args)
    )
    |> runtime.command_arg (
        runtime.new_arg ##(get_command_args () .trace_level)
        |> runtime.arg_short 't'
        |> runtime.arg_long ##(get_command_args () .trace_level)
        |> real runtime.arg_union `trace_level ignore
    )

()



## get_subcommands_args

In [ ]:
inl get_subcommands_args () =
    {
        rust = "rust", {
            deps = "deps"
        }
    }

()



## get_subcommands

In [ ]:
let get_subcommands () =
    ##"command"
    |> runtime.new_command
    |> runtime.command_subcommand_required true
    |> runtime.command_subcommand (
        ##(get_subcommands_args () .rust |> fst)
        |> runtime.new_command
        |> runtime.command_arg (
            runtime.new_arg ##((get_subcommands_args () .rust |> snd).deps)
            |> runtime.arg_short 'd'
            |> runtime.arg_long ##((get_subcommands_args () .rust |> snd).deps)
            |> runtime.arg_value_names ;[ ##"NAME"; ##"VERSION" ]
            |> runtime.arg_num_args_range (
                runtime.new_value_range
                    (am'.Start (1i32 |> unativeint))
                    (am'.End id)
            )
            |> runtime.arg_action runtime.Append
        )
    )

()



## get_workspace_cargo_toml_content

In [ ]:
inl get_workspace_cargo_toml_content () : string =
    $'$"""[workspace]\nresolver = "2"\nmembers = ["packages/*"]\n\n[workspace.dependencies]\nfable_library_rust = {{ path = \"../../../../../lib/rust/fable/fable_modules/fable-library-rust\", default-features = false, features = [\"static_do_bindings\", \"datetime\", \"guid\", \"threaded\"] }}\ninline_colorization = \"~0.1\"\n\n"""'

()



## get_cargo_toml_content

In [ ]:
inl get_cargo_toml_content { hash deps } : string =
    $'$"""[package]\nname = \"spiral_builder_{!hash}\"\nversion = "0.0.1"\nedition = "2021"\n\n[dependencies]\nfable_library_rust = {{ workspace = true }}\ninline_colorization = {{ workspace = true }}\n{!deps}\n\n[[bin]]\nname = \"spiral_builder_{!hash}\"\npath = \"spiral_eval.rs\" """'

()



## run

In [ ]:
let run { file package_dir args }
    : async.future_pin (resultm.result' string string) =

    inl file = file |> file_system.get_full_path
    inl package_dir = package_dir |> file_system.get_full_path
    inl hash = package_dir |> file_system.get_file_name

    trace Debug
        fun () => $'"spiral_builder.run"'
        fun () => $'$"file: {!file} / package_dir: {!package_dir} / hash: {!hash} / args: {!args} / {!_locals ()}"'

    fun () =>
        inl subcommands =
            get_subcommands ()
            |> runtime.command_get_matches_from ($'$"spiral_builder {!args}"' |> runtime.split_args)

        inl cargo_toml_content =
            match subcommands |> runtime.matches_subcommand |> optionm'.unbox with
            | Some (subcommand, arg_matches)
                when sm'.from_std_string subcommand = (get_subcommands_args () .rust |> fst)
                =>
                inl deps =
                    arg_matches
                    |> runtime.matches_get_many ((get_subcommands_args () .rust |> snd).deps)
                    |> optionm'.unbox
                    |> optionm'.default_value (;[] |> am'.to_vec)
                
                inl len = deps |> am'.vec_len |> i32
                
                inl deps =
                    deps
                    |> am'.vec_map fun dep =>
                        inl dep = dep |> sm'.from_std_string
                        if dep |> sm'.contains "="
                        then dep
                        else $'$"{!dep}=\\"*\\""'
                    |> am'.from_vec
                    |> fun x => x : _ i32 _
                    |> seq.of_array'
                    |> sm'.concat "\n"

                inl cargo_toml_content = get_cargo_toml_content { hash deps }
                cargo_toml_content

            | Some (subcommand, arg_matches) =>
                trace Debug
                    fun () => $'"spiral_builder.run"'
                    fun () => $'$"subcommand: {!subcommand} / {!_locals ()}"'

                inl cargo_toml_content = get_cargo_toml_content { hash deps = "" }
                cargo_toml_content
            | _ => ""

        inl workspace_cargo_toml_content = get_workspace_cargo_toml_content ()
        
        inl result =
            ;[
                "cargo_toml_content", cargo_toml_content
                "workspace_cargo_toml_content", workspace_cargo_toml_content
            ]
            |> am'.to_vec
            |> am'.vec_map' fun k, v =>
                new_pair (sm'.to_std_string k) (sm'.to_std_string v)
            |> mapm.b_tree_map_from_vec_pairs

        trace Debug
            fun () => $'"spiral_builder.run"'
            fun () => $'$"result: %A{!result} / {!_locals ()}"'
        
        result
        |> sm'.serialize
        |> resultm.map_error' (sm'.format' >> sm'.from_std_string)
        |> resultm.map' sm'.from_std_string
    |> async.future_init (4, 3) 1

()



In [ ]:
//// test
//// print_code=false
//// trace=false
///! rust -d clap futures-lite regex serde_json

types ()
run {
    file = "file"
    package_dir = "package_dir"
    args = "rust"
}
|> async.block_on
|> resultm.unwrap'
|> _assert_eq (
    ;[
        "cargo_toml_content",
        get_cargo_toml_content { hash = "package_dir"; deps = "" }

        "workspace_cargo_toml_content",
        get_workspace_cargo_toml_content ()
    ]
    |> am'.to_vec
    |> am'.vec_map' fun k, v =>
        new_pair (sm'.to_std_string k) (sm'.to_std_string v)
    |> mapm.b_tree_map_from_vec_pairs
    |> sm'.serialize
    |> resultm.unwrap'
    |> sm'.from_std_string
)

00:00:00   debug #1 spiral_builder.run / file: C:\home\git\polyglot\apps\spiral\builder\file / package_dir: C:\home\git\polyglot\apps\spiral\builder\package_dir / hash: package_dir / args: rust
00:00:00   debug #2 spiral_builder.run / result: {"cargo_toml_content": "[package]\nname = \"spiral_builder_package_dir\"\nversion = \"0.0.1\"\nedition = \"2021\"\n\n[dependencies]\nfable_library_rust = { workspace = true }\ninline_colorization = { workspace = true }\n\n\n[[bin]]\nname = \"spiral_builder_package_dir\"\npath = \"spiral_eval.rs\" ", "workspace_cargo_toml_content": "[workspace]\nresolver = \"2\"\nmembers = [\"packages/*\"]\n\n[workspace.dependencies]\nfable_library_rust = { path = \"../../../../../lib/rust/fable/fable_modules/fable-library-rust\", default-features = false, features = [\"static_do_bindings\", \"datetime\", \"guid\", \"threaded\"] }\ninline_colorization = \"~0.1\"\n\n"}
assert_eq / actual: "{"cargo_toml_content":"[package]\nname = \"spiral_builder_package_dir\"\nvers

In [ ]:
//// test
//// print_code=false
//// trace=false
///! rust -d clap futures-lite regex serde_json

types ()
run {
    file = "file"
    package_dir = "package_dir"
    args = "rust --deps a b=\'1\'"
}
|> async.block_on
|> resultm.unwrap'
|> _assert_eq (
    ;[
        "cargo_toml_content",
        get_cargo_toml_content { hash = "package_dir"; deps = "a=\"*\"\nb=\'1\'" }

        "workspace_cargo_toml_content",
        get_workspace_cargo_toml_content ()
    ]
    |> am'.to_vec
    |> am'.vec_map' fun k, v =>
        new_pair (sm'.to_std_string k) (sm'.to_std_string v)
    |> mapm.b_tree_map_from_vec_pairs
    |> sm'.serialize
    |> resultm.unwrap'
    |> sm'.from_std_string
)

00:00:00   debug #1 spiral_builder.run / file: C:\home\git\polyglot\apps\spiral\builder\file / package_dir: C:\home\git\polyglot\apps\spiral\builder\package_dir / hash: package_dir / args: rust --deps a b='1'
00:00:00   debug #2 spiral_builder.run / result: {"cargo_toml_content": "[package]\nname = \"spiral_builder_package_dir\"\nversion = \"0.0.1\"\nedition = \"2021\"\n\n[dependencies]\nfable_library_rust = { workspace = true }\ninline_colorization = { workspace = true }\na=\"*\"\nb='1'\n\n[[bin]]\nname = \"spiral_builder_package_dir\"\npath = \"spiral_eval.rs\" ", "workspace_cargo_toml_content": "[workspace]\nresolver = \"2\"\nmembers = [\"packages/*\"]\n\n[workspace.dependencies]\nfable_library_rust = { path = \"../../../../../lib/rust/fable/fable_modules/fable-library-rust\", default-features = false, features = [\"static_do_bindings\", \"datetime\", \"guid\", \"threaded\"] }\ninline_colorization = \"~0.1\"\n\n"}
assert_eq / actual: "{"cargo_toml_content":"[package]\nname = \"spira

## tests

In [ ]:
inl tests () =
    rust.run_tests [
        "verify_app", fun _ =>
            get_command () |> runtime.command_debug_assert
    ]

()



## main

In [ ]:
///! 

inl main (args : array_base string) =
    inl trace_state = get_trace_state ()
    inl trace_state_level = trace_state.level
    trace_state_level <- Info

    trace Debug
        fun () => $'$"spiral_builder.main"'
        fun () => $'$"args: {!args} / {!_locals ()}"'

    inl command = get_command ()
    inl arg_matches = command |> runtime.command_get_matches

    inl file =
        arg_matches
        |> runtime.matches_get_one (get_command_args () .file)
        |> optionm'.unbox
        |> optionm.value
        |> sm'.from_std_string

    inl package_dir =
        arg_matches
        |> runtime.matches_get_one (get_command_args () .package_dir)
        |> optionm'.unbox
        |> optionm.value
        |> sm'.from_std_string

    inl args =
        arg_matches
        |> runtime.matches_get_one (get_command_args () .args)
        |> optionm'.unbox
        |> optionm.map sm'.from_std_string
        |> optionm'.default_value ""

    inl trace_level =
        arg_matches
        |> runtime.matches_get_one (get_command_args () .trace_level)
        |> optionm'.unbox
        |> optionm.map sm'.from_std_string

    match trace_level with
    | Some trace_level => trace_state_level <- trace_level |> of_string
    | None => ()

    inl result =
        run { file package_dir args }
        |> async.block_on
        |> resultm.unwrap'

    if *trace_state_level = Info
    then result |> console.write_line

    0i32

inl main () =
    types ()
    $"let tests () = !tests ()" : ()
    $"let main args = !main args" : ()

.fsx:
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("std::env::VarError")>]
#endif
type std_env_VarError = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("std::fs::File")>]
#endif
type std_fs_File = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("std::fs::FileType")>]
#endif
type std_fs_FileType = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("std::path::Display")>]
#endif
type std_path_Display = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("std::path::Path")>]
#endif
type std_path_Path = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("std::path::PathBuf")>]
#endif
type std_path_PathBuf = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("async_walkdir::DirEntry")>]
#endif
type async_walkdir_DirEntry = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("async_walkdir::Filtering")>]
#endif
type async_walkdir_Filtering = class end
#if FABLE_COMPILER
[<Fable.C